## Revisando as possiveis Tabelas

In [1]:
# abrir bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
%matplotlib inline

## Importando EDAs Prontos

In [ ]:
basics = pd.read_csv('C:/Users/afilg/OneDrive/DATA SCIENCE/Aulas - Originais/Projeto_Integrador/basics.csv')

In [ ]:
budgets = pd.read_csv('C:/Users/afilg/OneDrive/DATA SCIENCE/Aulas - Originais/Projeto_Integrador/budgets.csv')

In [ ]:
rt_info_critica = pd.read_csv('C:/Users/afilg/OneDrive/DATA SCIENCE/Aulas - Originais/Projeto_Integrador/rt_info_critica.csv')

In [ ]:
oscar = pd.read_csv('C:/Users/afilg/OneDrive/DATA SCIENCE/Aulas - Originais/Projeto_Integrador/oscar.csv')

In [ ]:
crew = pd.read_csv('C:/Users/afilg/OneDrive/DATA SCIENCE/Aulas - Originais/Projeto_Integrador/crew.csv')

In [ ]:
principals = pd.read_csv('C:/Users/afilg/OneDrive/DATA SCIENCE/Aulas - Originais/Projeto_Integrador/principals.csv')

## Definindo EDA Functions

### Diretores

In [ ]:
def directors_ratingAverage(diretor, ano, df):
    x = df[(df.startYear < ano) & ((df.directors_1 == diretor)|(df.directors_2 == diretor))]
    z = x['rating_num'].sum()/x['numVotes'].sum()
    return z.round(2)

In [ ]:
def directors_numerofilmes(diretor, ano, df):
    x = df[(df.startYear < ano) & ((df.directors_1 == diretor)|(df.directors_2 == diretor))]
    z = x.shape
    return z[0]

### Roteristas

In [ ]:
def writers_numerofilmes(writer, ano, df):
    x = df[(df.startYear < ano) & ((df.writers_1 == writer)|(df.writers_2 == writer))]
    z = len(x)
    return z

In [ ]:
def writers_ratingAverage(writer, ano, df):
    x = df[(df.startYear < ano) & ((df.writers_1 == writer)|(df.writers_2 == writer))]
    z = x['rating_num'].sum()/x['numVotes'].sum()
    return z

### Atores

In [ ]:
def ator_numerofilmes(nconst, startYear):
    x = principals_temp[(principals_temp.startYear < startYear) & (principals_temp.nconst == nconst)]
    z = len(x)
    return z

In [ ]:
def ator_ratingAverage(ator, ano, df):
    x = df[(df.startYear < ano) & (df.nconst == ator)]
    z = x['rating_num'].sum()/x['numVotes'].sum()

## Juntando as tabelas

### Tabelas Base

In [ ]:
budgets.shape

In [ ]:
basics_budget = basics.join(budgets.set_index('movieyear_id'), on = 'movieyear', how = 'inner')

In [ ]:
basics_budget.shape

In [ ]:
basics_budget_rt = basics_budget.join(rt_info_critica.set_index('movieyear_id'), on = 'movieyear', how = 'inner')

In [ ]:
basics_budget_rt.shape

In [ ]:
basics_budget_rt_oscar = basics_budget_rt.join(oscar.set_index('movieyear_id'), on = 'movieyear')

### Crew

In [ ]:
basics_budget_rt_oscar_crew = basics_budget_rt_oscar.join(crew.set_index('tconst'), on = 'tconst', how = 'inner', rsuffix = "_2")

In [ ]:
basics_budget_rt_oscar_crew.head(2).T

In [ ]:
basics_budget_rt_oscar_crew = basics_budget_rt_oscar_crew.drop(['startYear_2', 'averageRating_2', 'numVotes_2'], axis = 1)

In [ ]:
crew_redux = crew.loc[:,['directors_2','directors_1','startYear']]

In [ ]:
tqdm.pandas()

basics_budget_rt_oscar_crew['directors_1_films'] = basics_budget_rt_oscar_crew.loc[:,['directors_1','startYear']].progress_apply(lambda x: directors_numerofilmes(x['directors_1'], x['startYear'],crew_redux ),axis = 1)

In [ ]:
tqdm.pandas()

basics_budget_rt_oscar_crew['directors_2_films'] = basics_budget_rt_oscar_crew.loc[:,['directors_2','startYear']].progress_apply(lambda x: directors_numerofilmes(x['directors_2'], x['startYear'],crew_redux ),axis = 1)

In [ ]:
crew_redux = crew.loc[:,['directors_2','directors_1','startYear', 'rating_num','numVotes']]

In [ ]:
tqdm.pandas()

basics_budget_rt_oscar_crew['directors_1_ratingAverage'] = basics_budget_rt_oscar_crew.loc[:,['directors_1','startYear']].progress_apply(lambda x: directors_ratingAverage(x['directors_1'], x['startYear'],crew_redux),axis = 1)

In [ ]:
tqdm.pandas()

basics_budget_rt_oscar_crew['directors_2_ratingAverage'] = basics_budget_rt_oscar_crew.loc[:,['directors_2','startYear']].progress_apply(lambda x: directors_ratingAverage(x['directors_2'], x['startYear'],crew_redux),axis = 1)

In [ ]:
crew_redux = crew.loc[:,['writers_2','writers_1','startYear']]

In [ ]:
tqdm.pandas()

basics_budget_rt_oscar_crew['writer_1_films'] = basics_budget_rt_oscar_crew.loc[:,['writers_1','startYear']].progress_apply(lambda x: writers_numerofilmes(x['writers_1'], x['startYear'],crew_redux),axis = 1)

In [ ]:
tqdm.pandas()

basics_budget_rt_oscar_crew['writer_2_films'] = basics_budget_rt_oscar_crew.loc[:,['writers_2','startYear']].progress_apply(lambda x: writers_numerofilmes(x['writers_2'], x['startYear'],crew_redux),axis = 1)

In [ ]:
crew_redux = crew.loc[:,['writers_1','writers_2','startYear', 'rating_num','numVotes']]

In [ ]:
tqdm.pandas()

basics_budget_rt_oscar_crew['writer_1_ratingAverage'] = basics_budget_rt_oscar_crew.loc[:,['writers_1','startYear']].progress_apply(lambda x: writers_ratingAverage(x['writers_1'], x['startYear'],crew_redux),axis = 1)

In [ ]:
tqdm.pandas()

basics_budget_rt_oscar_crew['writer_2_ratingAverage'] = basics_budget_rt_oscar_crew.loc[:,['writers_2','startYear']].progress_apply(lambda x: writers_ratingAverage(x['writers_2'], x['startYear'],crew_redux),axis = 1)

In [ ]:
basics_budget_rt_oscar_crew.head().T

### Principais

In [ ]:
temp_princ_util = principals.join(basics_budget_rt_oscar_crew.set_index('tconst'), on = 'tconst', how = 'inner')

In [ ]:
tqdm.pandas()

temp_princ_util['numFilmes'] = temp_princ_util.progress_apply(lambda x: ator_numerofilmes(x['nconst'], x['startYear']),axis = 1)

In [ ]:
tqdm.pandas()
temp_princ_util['AvgRating'] = temp_princ_util.progress_apply(lambda x: ator_ratingAverage(x['nconst'], x['startYear'],principals_temp),axis = 1)

In [ ]:
final_princ = pd.DataFrame(pd.pivot_table(temp_princ_util, 
                      values=('numFilmes','AvgRating'), 
                      index='tconst', 
                      aggfunc=np.mean))

In [ ]:
final_princ.head()

In [ ]:
num_crew = pd.DataFrame(pd.pivot_table(principals, 
                      values=('num_crew'), 
                      index='tconst', 
                      aggfunc=np.mean))

In [ ]:
final_princ = final_princ.join(num_crew)

In [ ]:
final_princ

In [ ]:
basics_budget_rt_crew_actors = basics_budget_rt_oscar_crew.join(final_princ.set_index('movieyear_id'), on = 'movieyear', how = 'inner')

In [ ]:
basics_budget_rt_crew_actors.to_csv('final.csv')